In [1]:
print("Fala")

Fala


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("../data/pessoas_completo.csv")

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1030380 entries, 0 to 1030386
Data columns (total 19 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   id_sinistro               1030380 non-null  int64  
 1   municipio                 1030380 non-null  object 
 2   tipo_via                  1030380 non-null  object 
 3   tipo_veiculo_vitima       58157 non-null    object 
 4   sexo                      1030380 non-null  object 
 5   idade                     1030380 non-null  int64  
 6   data_obito                58157 non-null    object 
 7   gravidade_lesao           1030380 non-null  object 
 8   tipo_vitima               1030380 non-null  object 
 9   faixa_etaria_demografica  1030380 non-null  object 
 10  faixa_etaria_legal        1030380 non-null  object 
 11  profissao                 1030380 non-null  object 
 12  data_sinistro             1030380 non-null  object 
 13  ano_sinistro              1

In [9]:
df.describe()

,id_sinistro,idade,ano_sinistro,mes_sinistro,dia_sinistro,ano_obito,mes_obito,dia_obito
count,1.030387e+06,1.013700e+06,1.030387e+06,1.030387e+06,1.030387e+06,58157.000000,58157.000000,58157.000000
mean,1.943842e+06,3.444966e+01,2.021658e+03,6.499242e+00,1.560665e+01,2019.789501,6.497257,15.580119
std,3.947906e+05,1.510009e+01,2.027790e+00,3.376548e+00,8.735699e+00,3.169683,3.378640,8.779323
min,1.256212e+06,0.000000e+00,2.014000e+03,1.000000e+00,1.000000e+00,2015.000000,1.000000,1.000000
25%,1.602038e+06,2.300000e+01,2.020000e+03,4.000000e+00,8.000000e+00,2017.000000,4.000000,8.000000
50%,1.943348e+06,3.100000e+01,2.022000e+03,6.000000e+00,1.600000e+01,2020.000000,6.000000,16.000000
75%,2.278146e+06,4.300000e+01,2.023000e+03,9.000000e+00,2.300000e+01,2023.000000,9.000000,23.000000
max,2.644715e+06,2.700000e+02,2.025000e+03,1.200000e+01,3.100000e+01,2025.000000,12.000000,31.000000


In [5]:
df.head()

,id_sinistro,municipio,tipo_via,tipo_veiculo_vitima,sexo,idade,data_obito,gravidade_lesao,tipo_de vítima,faixa_etaria_demografica,...,profissao,data_sinistro,ano_sinistro,mes_sinistro,dia_sinistro,ano_mes_sinistro,ano_obito,mes_obito,dia_obito,ano_mes_obito
0,2463323,SAO PAULO,VIAS MUNICIPAIS,PEDESTRE,MASCULINO,51.0,01/01/2015,FATAL,PEDESTRE,50 a 54,...,NaN,01/01/2015,2015,1,1,2015/01,2015.0,1.0,1.0,2015/01
1,2465127,SAO PAULO,VIAS MUNICIPAIS,PEDESTRE,MASCULINO,89.0,03/01/2015,FATAL,PEDESTRE,85 a 89,...,NaN,03/01/2015,2015,1,3,2015/01,2015.0,1.0,3.0,2015/01
2,2462123,SAO PAULO,VIAS MUNICIPAIS,PEDESTRE,MASCULINO,67.0,05/01/2015,FATAL,PEDESTRE,65 a 69,...,NaN,05/01/2015,2015,1,5,2015/01,2015.0,1.0,5.0,2015/01
3,2506869,MOGI GUACU,VIAS MUNICIPAIS,NAO DISPONIVEL,NAO DISPONIVEL,44.0,09/01/2015,FATAL,NAO DISPONIVEL,40 a 44,...,NaN,09/01/2015,2015,1,9,2015/01,2015.0,1.0,9.0,2015/01
4,2490194,COTIA,RODOVIAS,AUTOMOVEL,MASCULINO,28.0,11/01/2015,FATAL,CONDUTOR,25 a 29,...,NaN,11/01/2015,2015,1,11,2015/01,2015.0,1.0,11.0,2015/01


In [17]:
df.count()

id_sinistro                 1030380
municipio                   1030380
tipo_via                    1030380
tipo_veiculo_vitima           58157
sexo                        1030380
idade                       1030380
data_obito                    58157
gravidade_lesao             1030380
tipo_vitima                  992631
faixa_etaria_demografica    1030380
faixa_etaria_legal          1030380
profissao                   1030380
data_sinistro               1030380
ano_sinistro                1030380
mes_sinistro                1030380
dia_sinistro                1030380
ano_obito                     58157
mes_obito                     58157
dia_obito                     58157
dtype: int64

In [27]:
df.isna().sum()

id_sinistro                      0
municipio                        0
tipo_via                         0
tipo_veiculo_vitima         972223
sexo                             0
idade                            0
data_obito                  972223
gravidade_lesao                  0
tipo_vitima                      0
faixa_etaria_demografica         0
faixa_etaria_legal               0
profissao                        0
data_sinistro                    0
ano_sinistro                     0
mes_sinistro                     0
dia_sinistro                     0
ano_obito                   972223
mes_obito                   972223
dia_obito                   972223
dtype: int64

# Análise do CSV _pessoas_completo_

In [34]:
df["mes_obito"].isna().sum()

972223

In [32]:
df["mes_obito"].value_counts()

7.0     5431
5.0     5391
6.0     5199
3.0     5102
12.0    4879
4.0     4873
10.0    4812
8.0     4704
9.0     4688
1.0     4487
11.0    4304
2.0     4287
Name: mes_obito, dtype: int64

## Removendo colunas e linhas
Alguns dados da tabela estão com valores desnecessários pra análise

Remover colunas **ano_mes_obito** e **ano_mes_sinistro** pois são redundantes

In [9]:
df = df.drop(columns=["ano_mes_sinistro", "ano_mes_obito"])

Remover linhas com valor nulo nas colunas **municipio** e **tipo_via** 

In [13]:
df = df.dropna(subset=["municipio", "tipo_via"])

Verificação se todos os acidentes que são fatais contém data do óbito

In [11]:
((df["gravidade_lesao"] == "FATAL") & (df["data_obito"].isna())).sum()

0

## Renomeação e conversão de colunas e valores de linhas
Muitos dados precisam ser tratados para poderem ser analisados mais apropriadamente

Renomeando coluna **tipo_de vítima** para **tipo_vitima**

In [8]:
df = df.rename(columns={"tipo_de vítima": "tipo_vitima"})

Adicionando idade média nos valores nulos da coluna **idade** e renomeando valores nulos das colunas **profissao** e **tipo_vitima**

In [26]:
df["idade"] = df["idade"].fillna(df["idade"].mean())
df["profissao"] = df["profissao"].fillna("DESCONHECIDA")
df["tipo_vitima"] = df["tipo_vitima"].fillna("NAO DISPONIVEL")

Renomeando valores _nao disponivel_ da coluna **sexo** para _ignorado_

In [15]:
df["sexo"] = df["sexo"].replace({"NAO DISPONIVEL": "IGNORADO"})

Mudando tipo de dado das colunas **idade, mes_obito** e **mes_sinistro** de `float64` para `int64`

In [35]:
df["idade"] = df["idade"].astype("int64")
df["mes_sinistro"] = df["mes_sinistro"].astype("int64")
df["mes_obito"] = df["mes_obito"].notna().astype("int64")

Criar CSV com o dataset

In [ ]:
df.to_csv("pessoas_completo_limpo.csv", index=False)